## Bad Performance on DSC

In [4]:
import os
import pandas as pd

# Path to the root folder (adjust as needed)
models_root = 'final_14dec2024_results/jhh_results - table 2 in paper'

bad_performance_threshold = 0.3

# Dictionary to store results
low_performance_dict = {}

# Loop through each model folder
for model_name in os.listdir(models_root):
    model_path = os.path.join(models_root, model_name)
    dsc_path = os.path.join(model_path, 'dsc.csv')

    if os.path.isfile(dsc_path):
        dsc_df = pd.read_csv(dsc_path)

        # Initialize dictionary for this model
        model_results = {}

        for organ in dsc_df.columns[1:]:  # Skip 'name' column
            if organ in dsc_df:
                low_perf_ids = dsc_df[dsc_df[organ] < bad_performance_threshold]['name'].tolist()
                model_results[organ] = low_perf_ids

        low_performance_dict[model_name] = model_results

In [6]:
# Now count how many low-performance cases there are per model per organ
counts_dict = {
    model: {organ: len(ct_ids) for organ, ct_ids in organs.items()}
    for model, organs in low_performance_dict.items()
}

# Convert to DataFrame for visualization
counts_df = pd.DataFrame.from_dict(counts_dict, orient='index').fillna(0).astype(int)

# Display or save the DataFrame
counts_df

,aorta,gall_bladder,kidney_left,kidney_right,liver,pancreas,postcava,spleen,stomach
UNETR,139,235,44,40,11,147,76,37,16
U-Net_CLIP,32,60,25,18,7,43,12,14,7
Diff-UNet,15,98,30,28,12,65,15,26,12
NexToU,17,146,38,35,15,93,16,33,18
Swin_UNETR_CLIP,25,95,26,24,9,46,19,17,13
STU-Net H,12,51,27,25,10,60,11,17,9
nnU-Net_ResEncL,13,65,26,25,11,63,12,22,10
SegVol,20,188,27,26,11,84,18,24,12
UNEST,137,278,35,37,11,85,51,21,16
MedNeXt,12,69,28,29,11,65,11,22,9
